#Modelo de Machine Learning em Python para Avaliação de Imóveis.
##Desenvolvido por Flávio Cardoso
##Utiliza alguns algoritmos de Machine Learnig para ao final escolher o que tem melhor performance.
##Depois utiliza um modelo de AutoML (H2o) para efeito de comparação com os modelos testatos anteriormente
##Utiliza como dados de entrada o banco de dados de 2016 a 2020, obtido através do trabalho de vistoria de imóveis pelos engenheiros credenciados à Caixa Econômica, que são compilados pela Caixa e devolvido aos mesmo engenheiros para uso em trabalho de avaliação para a Caixa.
##Utiliza também como dado de entrada um arquivo do imóvel a ser avaliado



##Instala bibliotecas necessárias

In [17]:
#!pip install h2o

#! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

##Importa as bibliotecas que serão utilizadas

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h2o
from h2o.automl import H2OAutoML
h2o.init()
import numpy as np
from sklearn.model_selection import train_test_split

from scipy import stats
from pandas_profiling import ProfileReport

from sklearn.ensemble import IsolationForest,RandomForestRegressor,GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor


from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor




Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,58 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_unknownUser_xr94ql
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


##Abre o arquivo contendo os dados do imóvel avaliando

In [19]:
path='/content/Terreno_Ago16-Set20+Pesquisa-Fortaleza-Python.xls'
tipo=path[9:13]

df = pd.read_excel(path,sheet_name='Dados')
#df=df.drop(['Endereço','Bairro','Complemento','Informante','Telefone','Municipio'],axis=1)
df


,Latitude,Longitude,Area,Testada,Oferta/Trans,Esq-SN,Renda Media Domic,Valor Unitario
0,-3.738982,-38.554722,108.0,4.50000,1,1,2468.55,1666.67
1,-3.798333,-38.538889,156.0,6.00000,2,1,3575.48,846.15
2,-3.835833,-38.587778,168.0,6.00000,2,1,1186.33,892.86
3,-3.736248,-38.556050,200.0,7.00000,1,1,3379.20,1125.00
4,-3.819016,-38.555054,200.0,8.00000,2,1,3069.60,740.00
...,...,...,...,...,...,...,...,...
99,-3.864366,-38.486186,14904.0,122.08194,2,2,953.23,100.64
100,-3.850029,-38.504641,16236.0,33.00000,1,1,1113.58,172.46
101,-3.832522,-38.582604,16532.0,133.00000,2,2,1082.50,211.71
102,-3.822003,-38.577955,29500.0,110.00000,2,2,732.72,328.81


##Abre o banco de dados de Imóveis, neste caso de casas. 

In [20]:
x_imovel=pd.read_excel('/content/Terreno_Ago16-Set20+Pesquisa-Fortaleza-Python.xls',sheet_name='Terreno')
#x_imovel=x_imovel.drop(['Endereço','Bairro','Complemento','Informante','Telefone','Municipio'],axis=1)
x_imovel.head()

,Latitude,Longitude,Area,Testada,Oferta/Trans,Esq-SN,Renda Media Domic,Valor Unitario
0,-3.7948,-38.469985,3200,80,1,2,1902,1666


##Informe o município onde se localiza o imóvel avaliando e filtra o dataframe para conter apenas dados deste município

In [21]:
#municipio = "ITAITINGA"
#df=df[(df.Municipio == municipio)]
#df

##Converte casas decimais com vírgula para ponto de alguns atributos

In [22]:
df['Latitude']=df['Latitude'].astype(str)
df['Latitude']=df['Latitude'].str.replace(',','.')
df['Latitude']=df['Latitude'].astype(float)

df['Longitude']=df['Longitude'].astype(str)
df['Longitude']=df['Longitude'].str.replace(',','.')
df['Longitude']=df['Longitude'].astype(float)
df

,Latitude,Longitude,Area,Testada,Oferta/Trans,Esq-SN,Renda Media Domic,Valor Unitario
0,-3.738982,-38.554722,108.0,4.50000,1,1,2468.55,1666.67
1,-3.798333,-38.538889,156.0,6.00000,2,1,3575.48,846.15
2,-3.835833,-38.587778,168.0,6.00000,2,1,1186.33,892.86
3,-3.736248,-38.556050,200.0,7.00000,1,1,3379.20,1125.00
4,-3.819016,-38.555054,200.0,8.00000,2,1,3069.60,740.00
...,...,...,...,...,...,...,...,...
99,-3.864366,-38.486186,14904.0,122.08194,2,2,953.23,100.64
100,-3.850029,-38.504641,16236.0,33.00000,1,1,1113.58,172.46
101,-3.832522,-38.582604,16532.0,133.00000,2,2,1082.50,211.71
102,-3.822003,-38.577955,29500.0,110.00000,2,2,732.72,328.81


In [23]:
#df=df.drop(['Endereço','Bairro'],axis=1)
#df
y=df['Valor Unitario'].values

##Faz uma EAD (Análise Exploratória dos dados) inicial

In [24]:
#profile = ProfileReport(df)
#profile.to_notebook_iframe()


##Localiza e elimina valores nulos

In [25]:
df.dropna(axis=0,inplace=True)
df.isnull().sum()

Latitude             0
Longitude            0
Area                 0
Testada              0
Oferta/Trans         0
Esq-SN               0
Renda Media Domic    0
Valor Unitario       0
dtype: int64

##Faz a detecção automática e elimina os outliers com o método Local Outlier Factor


In [ ]:
# identify outliers in the training dataset
lof = LocalOutlierFactor()

colunas = df.columns
df=df.values

yhat = lof.fit_predict(df)
# select all rows that are not outliers
mask = yhat != -1
df, y = df[mask, :], y[mask]
# summarize the shape of the updated training dataset
print(df.shape, y.shape)

df=pd.DataFrame(df)
df.columns = colunas

##Faz nova EAD para verificar a redução dos outliers

In [26]:
#profile = ProfileReport(df)
#profile.to_notebook_iframe()


##Identifica e elimina os atributos que possuem alta correlação entre si

In [ ]:
#Identify Highly Correlated Features
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.80
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]
#Drop Marked Features
# Drop features 
df.drop(df[to_drop], axis=1,inplace=True)
df

##Equipara os atributos do imóvel avaliando aos atributos do dataframe

In [27]:
x_imovel.drop(x_imovel[to_drop], axis=1,inplace=True)

x_imovel

,Latitude,Longitude,Area,Testada,Oferta/Trans,Esq-SN,Renda Media Domic,Valor Unitario
0,-3.7948,-38.469985,3200,80,1,2,1902,1666


##Separa os dados em dados de treino e teste

In [28]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['Valor Unitario'],axis=1), y, test_size = 0.3,random_state = 0)


##Cria uma função que treina e calcula os scores para cada modelo utilizado

In [29]:

def treina(name,model):
  model.fit(x_train, y_train)
  yhat = model.predict(x_test)

  mae = mean_absolute_error(y_test, yhat)
  r2=r2_score(y_test,yhat)
  y_imovel = model.predict(x_imovel)

  scores = [name,mae,r2,y_imovel]
  score_map.append(scores)


##Define os modelos que serão empregados e faz um loop na função acima para cada modelo e em seguida mostra os scores obtidos para cada um

In [30]:
models = [('L.Regression',LinearRegression()),('R.Forest',RandomForestRegressor()),('KNN', 
      KNeighborsRegressor()),('D. Tree', DecisionTreeRegressor()),('G.B.Regression' , GradientBoostingRegressor())]

score_map=[]
colunas = ['Model','mae','R2','Valor Unitario']

score_map.append(colunas)

for name, model in models:
  #print(name,model)
  treina(name,model)


score_map = pd.DataFrame(score_map[1:], columns=score_map[0])
best_mae_map = score_map.sort_values(by=['mae'],ascending=True)
best_mae_map
best_r2_map = score_map.sort_values(by=['R2'],ascending=False)
best_r2_map

ValueError: ignored

##Mostra o gráfico com os scores 

In [ ]:

sns.factorplot("Model", "R2", col="mae", data=score_map, kind="bar");


##Para este caso, tanto para a métrica R2 como para a métrica MAE, a melhor performance foi obtida com o modelo Random Forest

In [ ]:
best_row = score_map.loc[score_map['Model']=='R.Forest']
best_Valor_Unit = best_row['Valor Unitario']
best_Valor_Unit
best_y_imovel = x_imovel['Area']*best_Valor_Unit.values

print("Valor Unitário: %6.0f"% (best_Valor_Unit))
print("Valor do imóvel: %6.0f"% (best_y_imovel))





##Compara o valor obtido acima com o modelo de autoML h2o

In [ ]:

df_h2o = h2o.H2OFrame(df)
x_imovel_h20 = h2o.H2OFrame(x_imovel)
df_h2o

In [ ]:
splits = df_h2o.split_frame(ratios = [0.7], seed = 0)
train = splits[0]
test = splits[1]



In [ ]:
y='Valor Unitario'



Run AutoML
Run AutoML, stopping after 60 seconds. The max_runtime_secs argument provides a way to limit the AutoML run by time. When using a time-limited stopping criterion, the number of models train will vary between runs. If different hardware is used or even if the same machine is used but the available compute resources on that machine are not the same between runs, then AutoML may be able to train more models on one run vs another.
The test frame is passed explicitly to the leaderboard_frame argument here, which means that instead of using cross-validated metrics, we use test set metrics for generating the leaderboard.

In [ ]:
#aml = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "House")
#aml.train(y = y, training_frame = train, leaderboard_frame = test)

For demonstration purposes, we will also execute a second AutoML run, this time providing the original, full dataset, df (without passing a leaderboard_frame). This is a more efficient use of our data since we can use 100% of the data for training, rather than 80% like we did above. This time our leaderboard will use cross-validated metrics.
Note: Using an explicit leaderboard_frame for scoring may be useful in some cases, which is why the option is available.

In [ ]:
aml2 = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "House")
aml2.train(y = y, training_frame = df_h2o)

Leaderboard
Next, we will view the AutoML Leaderboard. Since we specified a leaderboard_frame in the H2OAutoML.train() method for scoring and ranking the models, the AutoML leaderboard uses the performance on this data to rank the models.
After viewing the "powerplant_lb_frame" AutoML project leaderboard, we compare that to the leaderboard for the "powerplant_full_data" project. We can see that the results are better when the full dataset is used for training.
A default performance metric for each machine learning task (binary classification, multiclass classification, regression) is specified internally and the leaderboard will be sorted by that metric. In the case of regression, the default ranking metric is mean residual deviance. In the future, the user will be able to specify any of the H2O metrics so that different metrics can be used to generate rankings on the leaderboard.

Now we will view a snapshot of the top models. Here we should see the two Stacked Ensembles at or near the top of the leaderboard. Stacked Ensembles can almost always outperform a single model

In [ ]:
aml2.leaderboard.head()

In [ ]:
# To see best parameters
aml2.leader.params
#aml2.leader.params.keys()
#aml2.leader.params['metalearner_algorithm']

Predict Using Leader Model
If you need to generate predictions on a test set, you can make predictions on the "H2OAutoML" object directly, or on the leader model object.

In [ ]:

pred = aml2.predict(test)
pred.head()

If needed, the standard model_performance() method can be applied to the AutoML leader model and a test set to generate an H2O model performance object.

In [ ]:
test

In [ ]:

perf = aml2.leader.model_performance(test)
perf

In [ ]:
resultados=test
resultados['Vlr_prev']=pred

In [ ]:
resultados

In [ ]:

x_imovel_h20

In [ ]:
y_imovel = aml2.predict(x_imovel_h20)
y_imovel

In [ ]:


print("Valor Unitário: %6.0f"% (y_imovel.flatten()))

print("Valor do imóvel: %6.0f"% (y_imovel.flatten() * x_imovel['Area'][0]))

